In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import slideflow as sf
from scipy.spatial.transform import Rotation as R

def create_project_from_jpg(image_dir, root='data'):
    original_root = root
    index = 1
    while os.path.exists(root):
        root = f"{original_root}_{index}"
        index += 1

    os.makedirs(root)

    project = sf.create_project(
        root=root,
        slides=image_dir,  # JPEG 이미지 경로 사용
    )
    return project

def extract_tiles(project, tile_px=299, tile_um=302):
    project.extract_tiles(
        tile_px=tile_px,
        tile_um=tile_um,
        save_tiles=True,  # 타일 저장 활성화
        img_format='jpg'  # 'png'도 가능
    )

def lo_ransac_filter(matches, keypoints1, keypoints2):
    src_pts = np.float32([keypoints1[m[0]] for m in matches])
    dst_pts = np.float32([keypoints2[m[1]] for m in matches])

    model, inliers = cv2.estimateAffinePartial2D(
        src_pts, dst_pts, method=cv2.RANSAC, ransacReprojThreshold=5.0
    )
    return src_pts[inliers.ravel() == 1], dst_pts[inliers.ravel() == 1], model

def kabsch_algorithm(src_pts, dst_pts):
    centroid_src = np.mean(src_pts, axis=0)
    centroid_dst = np.mean(dst_pts, axis=0)

    src_centered = src_pts - centroid_src
    dst_centered = dst_pts - centroid_dst

    H = np.dot(src_centered.T, dst_centered)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = np.dot(Vt.T, U.T)

    t = centroid_dst - np.dot(R, centroid_src)
    return R, t

def match_tiles_with_keypoints(he_tiles, ki67_tiles, keypoints_he, keypoints_ki67, matches):
    matched_tiles = []

    for he_idx, he_tile in enumerate(he_tiles):
        best_match = None
        best_score = float('inf')
        best_R, best_t = None, None

        for ki_idx, ki67_tile in enumerate(ki67_tiles):
            matched_idx = matches[(matches['keypoint_idx_img0'] == he_idx) & 
                                  (matches['keypoint_idx_img1'] == ki_idx)]
            if matched_idx.empty:
                continue

            src_pts, dst_pts, _ = lo_ransac_filter(
                matched_idx.to_numpy(), keypoints_he, keypoints_ki67
            )
            if len(src_pts) < 4:
                continue

            R, t = kabsch_algorithm(src_pts, dst_pts)
            error = np.mean(np.linalg.norm(np.dot(R, src_pts.T).T + t - dst_pts, axis=1))

            if error < best_score:
                best_match = (he_tile, ki67_tile)
                best_score = error
                best_R, best_t = R, t

        if best_match:
            matched_tiles.append((he_tile, best_match[1], best_R, best_t, best_score))

    return matched_tiles

def visualize_matched_tiles(matched_tiles, num_pairs=5):
    if len(matched_tiles) == 0:
        print("No matched tiles to visualize.")
        return

    num_pairs = min(num_pairs, len(matched_tiles))
    fig, axes = plt.subplots(num_pairs, 2, figsize=(10, 5 * num_pairs))

    if num_pairs == 1:
        axes = [axes]

    for i in range(num_pairs):
        he_tile, ki67_tile, _, _, score = matched_tiles[i]

        axes[i][0].imshow(he_tile)
        axes[i][0].set_title(f'H&E Tile (Score: {score:.4f}')
        axes[i][0].axis('off')

        axes[i][1].imshow(ki67_tile)
        axes[i][1].set_title(f'KI-67 Tile')
        axes[i][1].axis('off')

    plt.tight_layout()
    plt.show()

def tile_matching_pipeline(keypoints_he_path, keypoints_ki67_path, matches_path, image_dir, root='./project/data'):
    print("[1/4] SlideFlow 프로젝트 생성 중...")
    project = create_project_from_jpg(image_dir, root=root)

    print("[2/4] 타일링 진행...")
    try:
        extract_tiles(project, tile_px=299, tile_um=302)
    except Exception as e:
        print("Error during tile extraction:", e)
        return

    print("[3/4] 키포인트 및 매칭 정보 로드...")
    keypoints_he = np.load(keypoints_he_path, allow_pickle=True)
    keypoints_ki67 = np.load(keypoints_ki67_path, allow_pickle=True)
    matches = pd.read_csv(matches_path)

    print("[4/4] 타일 매칭 수행...")
    matched_tiles = match_tiles_with_keypoints([], [], keypoints_he, keypoints_ki67, matches)

    print("매칭 완료. 시각화 시작...")
    visualize_matched_tiles(matched_tiles)

keypoints_he_path = '/home/20223176/LightGlue/match/keypoints_37-HE_6-ki67_0.npy'
keypoints_ki67_path = '/home/20223176/LightGlue/match/keypoints_37-HE_6-ki67_1.npy'
matches_path = '/home/20223176/LightGlue/match/matches_scores_37-HE_6-ki67.csv'
image_dir = './images/'

tile_matching_pipeline(keypoints_he_path, keypoints_ki67_path, matches_path, image_dir)




ContextualVersionConflict: (triton 3.1.0 (/usr/gatoai/python/venv/3.8/lib/python3.8/site-packages), Requirement.parse('triton==2.2.0; platform_system == "Linux" and platform_machine == "x86_64" and python_version < "3.12"'), {'torch'})

In [ ]:
from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d

import torch

torch.set_grad_enabled(False)
from pathlib import Path

import pandas as pd
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

extractor = SuperPoint(max_num_keypoints=1024).eval().to(device)  # load the extractor
matcher = LightGlue(features="superpoint").eval().to(device)
feats0 = extractor.extract(image0.to(device))
feats1 = extractor.extract(image1.to(device))
matches01 = matcher({"image0": feats0, "image1": feats1})
feats0, feats1, matches01 = [
    rbd(x) for x in [feats0, feats1, matches01]
]  # remove batch dimension

kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]

In [4]:
import slideflow as sf

ContextualVersionConflict: (triton 3.1.0 (/usr/gatoai/python/venv/3.8/lib/python3.8/site-packages), Requirement.parse('triton==2.2.0; platform_system == "Linux" and platform_machine == "x86_64" and python_version < "3.12"'), {'torch'})

In [8]:

ContextualVersionConflict: (triton 3.1.0 (/usr/gatoai/python/venv/3.8/lib/python3.8/site-packages), Requirement.parse('triton==2.2.0; platform_system == "Linux" and platform_machine == "x86_64" and python_version < "3.12"'), {'torch'})from tifffile import TiffFile, TiffWriter

# 원본 TIFF 파일 경로
input_tiff = "./tiff_images/original_6-ki67.tiff"

# 수정된 TIFF 파일 경로
output_tiff = "./tiff_images/original_6-ki67_fixed.tiff"

# 픽셀당 마이크론 값 (예: 0.25 microns per pixel)
microns_per_pixel = 0.25

# Resolution 계산
x_resolution = 25400 / microns_per_pixel
y_resolution = 25400 / microns_per_pixel

# TIFF 수정
with TiffFile(input_tiff) as tif:
    image = tif.pages[0].asarray()  # 이미지 데이터 읽기

with TiffWriter(output_tiff, bigtiff=True) as tif:
    tif.write(
        image,
        resolution=(x_resolution, y_resolution),
        resolutionunit="CENTIMETER"
    )

print(f"Updated TIFF saved to {output_tiff}")


Updated TIFF saved to ./tiff_images/original_6-ki67_fixed.tiff


In [10]:
project = sf.create_project(root='data4', slides='./tiff_images/')
project.extract_tiles(tile_px=299, tile_um=302)


[11:01:21] INFO     Setting up project at data4

           INFO     Creating project at data4...

           INFO     Saved dataset source MyProject to data4/datasets.json

           INFO     Wrote annotations file to data4/annotations.csv

           INFO     Column 'slide' missing in annotations.

...

           INFO     Generating PDF (this may take some time)...

Output()

Skipping CSV update; no extraction reports found.


Output()

{}

ContextualVersionConflict: (triton 3.1.0 (/usr/gatoai/python/venv/3.8/lib/python3.8/site-packages), Requirement.parse('triton==2.2.0; platform_system == "Linux" and platform_machine == "x86_64" and python_version < "3.12"'), {'torch'})